In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
# from tqdm.notebook import tqdm
from random import shuffle
import torch
from torch import nn
import math
from glob import glob
import sys
import shutil  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

%matplotlib inline

cuda


In [3]:
trainImages = sorted(glob('Cropped MIC dataset/Martian Inverted Channels' + '/*/images/*.jpg'))
if not os.path.exists('Cropped MIC dataset/TrainingData'):
  os.makedirs('Cropped MIC dataset/TrainingData')
if not os.path.exists('Cropped MIC dataset/TrainingData/TrainingPatches'):
  os.makedirs('Cropped MIC dataset/TrainingData/TrainingPatches')
if not os.path.exists('Cropped MIC dataset/TrainingData/TrainingPatches/images'):
  os.makedirs('Cropped MIC dataset/TrainingData/TrainingPatches/images')
if not os.path.exists('Cropped MIC dataset/TrainingData/TrainingPatches/masks'):
  os.makedirs('Cropped MIC dataset/TrainingData/TrainingPatches/masks')

In [4]:
testImages = sorted(glob('Cropped MIC dataset/Test data' + '/*/images/*.jpg'))
if not os.path.exists('Cropped MIC dataset/TestingData'):
  os.makedirs('Cropped MIC dataset/TestingData')
if not os.path.exists('Cropped MIC dataset/TestingData/TestingPatches'):
  os.makedirs('Cropped MIC dataset/TestingData/TestingPatches')
if not os.path.exists('Cropped MIC dataset/TestingData/TestingPatches/images'):
  os.makedirs('Cropped MIC dataset/TestingData/TestingPatches/images')
if not os.path.exists('Cropped MIC dataset/TestingData/TestingPatches/masks'):
  os.makedirs('Cropped MIC dataset/TestingData/TestingPatches/masks')

In [5]:
patchSize = 256

In [6]:
def makePatches(dirName, images):

  patchHeight = patchSize
  patchWidth = patchSize
  patchList = []

  ImageSizes = []
  for ImagePath in images:
    image = cv2.imread(ImagePath)

    mask_path = ImagePath.replace('images', 'masks').replace('.jpg', '.png')
    mask = cv2.imread(mask_path)

    image_array = np.array(image)
    imageSize = image_array.shape
    imageHeight = imageSize[0]
    imageWidth = imageSize[1]

    ImageSizes.append([ImagePath[len(ImagePath)-30:-4],[imageWidth,imageHeight]])


    for y in range (0,imageHeight,patchSize):
      for x in range (0,imageWidth,patchSize):
        if ((y+patchHeight > imageHeight) and (x+patchWidth > imageWidth)):
          pat = image [y:imageHeight,x:imageWidth]
          patch = np.pad(pat,((0,y+patchHeight-imageHeight),(0,x+patchWidth-imageWidth),(0,0)),'reflect')
          maskPatch = mask [y:imageHeight,x:imageWidth]
          maskPatch = np.pad(maskPatch,((0,y+patchHeight-imageHeight),(0,x+patchWidth-imageWidth),(0,0)),'reflect')
        elif ((y+patchHeight <= imageHeight) and (x+patchWidth > imageWidth)):
          pat = image [y:y+patchHeight,x:imageWidth]
          patch = np.pad(pat,((0,0),(0,x+patchWidth-imageWidth),(0,0)),'reflect')
          maskPatch = mask [y:y+patchHeight,x:imageWidth]
          maskPatch = np.pad(maskPatch,((0,0),(0,x+patchWidth-imageWidth),(0,0)),'reflect')
        elif ((y+patchHeight > imageHeight) and (x+patchWidth <= imageWidth)):
          pat = image [y:imageHeight,x:x+patchWidth]
          patch = np.pad(pat,((0,y+patchHeight-imageHeight),(0,0),(0,0)),'reflect')
          maskPatch = mask [y:imageHeight,x:x+patchWidth]
          maskPatch = np.pad(maskPatch,((0,y+patchHeight-imageHeight),(0,0),(0,0)),'reflect')
        else:
          patch = image [y:y+patchHeight,x:x+patchWidth]
          maskPatch = mask [y:y+patchHeight,x:x+patchWidth]

        patchName = ImagePath[len(ImagePath)-30:-4]+"_"+str(int(y/patchSize))+"_"+str(int(x/patchSize))
        cv2.imwrite(dirName+'/images/'+patchName+'.jpg', patch)
        cv2.imwrite(dirName+'/masks/'+patchName+'.png', maskPatch)
        patchList.append(patch)
  return ImageSizes

In [7]:
trainImages = sorted(glob('Cropped MIC dataset/Martian Inverted Channels' + '/*/images/*.jpg'))
testImages = sorted(glob('Cropped MIC dataset/Test data' + '/*/images/*.jpg'))
trainImageSizes = makePatches('Cropped MIC dataset/TrainingData/TrainingPatches', trainImages)
testImageSizes = makePatches('Cropped MIC dataset/TestingData/TestingPatches', testImages)

In [ ]:
print(len(trainImageSizes))
print(len(testImageSizes))
print(testImageSizes)

In [8]:
import os
import numpy as np
import torch
import torch.utils.data
import torchvision.transforms as transforms
import PIL
import random
from scipy import ndimage


class segDataset(torch.utils.data.Dataset):
    def __init__(self, root, training, transform=None):
        super(segDataset, self).__init__()
        self.root = root
        self.training = training
        self.transform = transform
        self.IMG_NAMES = sorted(glob(self.root + '/images/*.jpg'))
        self.BGR_classes = {'Background' : [ 0, 0, 0],
                            'Inverted Channel' : [ 0, 0, 255]} # in BGR

        self.bin_classes = ['Background', 'Inverted Channel']


    def __getitem__(self, idx):
        
        img_path = self.IMG_NAMES[idx]
        mask_path = img_path.replace('images', 'masks').replace('.jpg', '.png')

        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path)
        plt.imshow(mask)
        cls_mask = np.zeros(mask.shape)

        cls_mask[mask == self.BGR_classes['Background']] = self.bin_classes.index('Background')
        cls_mask[mask == self.BGR_classes['Inverted Channel']] = self.bin_classes.index('Inverted Channel')
        torch.set_printoptions(profile="full")
        #print("class mask",cls_mask)
        cls_mask = cls_mask[:,:,2] #removing nearby elements [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] -> [0,2,4,6,8]
        #print("---------------------------------------------------")
        #print(cls_mask)
        if self.training==True:
            if self.transform:
              image = transforms.functional.to_pil_image(image)
              image = self.transform(image)
              image = np.array(image)

        # image = cv2.resize(image, (128,128))/255.0
        # cls_mask = cv2.resize(cls_mask, (128,128))
        image = cv2.resize(image, (patchSize,patchSize))/255.0
        cls_mask = cv2.resize(cls_mask, (patchSize,patchSize)) 
        image = np.moveaxis(image, -1, 0)

        return torch.tensor(image).float(), torch.tensor(cls_mask, dtype=torch.int64), img_path


    def __len__(self):
        return len(self.IMG_NAMES)

In [9]:
testDataset = segDataset('Cropped MIC dataset/TestingData', training = False)
print(testDataset.IMG_NAMES)
len(testDataset)

[]


0

In [10]:
import os
import numpy as np
import torch
import torch.utils.data
import torchvision.transforms as transforms
import PIL
import random
from scipy import ndimage


class segTrainDataset(torch.utils.data.Dataset):
    def __init__(self, root, training, transform=None):
        super(segTrainDataset, self).__init__()
        self.root = root
        self.training = training
        self.transform = transform
        self.IMG_NAMES = sorted(glob(self.root + '/*/images/*.jpg'))
        self.BGR_classes = {'Background' : [ 0, 0, 0],
                            'Inverted Channel' : [ 0, 0, 255]} # in BGR

        self.bin_classes = ['Background', 'Inverted Channel']


    def __getitem__(self, idx):
        
        img_path = self.IMG_NAMES[idx]
        mask_path = img_path.replace('images', 'masks').replace('.jpg', '.png')

        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path)
        plt.imshow(mask)
        cls_mask = np.zeros(mask.shape)

        cls_mask[mask == self.BGR_classes['Background']] = self.bin_classes.index('Background')
        cls_mask[mask == self.BGR_classes['Inverted Channel']] = self.bin_classes.index('Inverted Channel')
        torch.set_printoptions(profile="full")
        #print("class mask",cls_mask)
        cls_mask = cls_mask[:,:,2] #removing nearby elements [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] -> [0,2,4,6,8]
        #print("---------------------------------------------------")
        #print(cls_mask)
        if self.training==True:
            if self.transform:
              image = transforms.functional.to_pil_image(image)
              image = self.transform(image)
              image = np.array(image)

            # 90 degree rotation
            if np.random.rand()<0.5:
              angle = np.random.randint(4) * 90
              image = ndimage.rotate(image,angle,reshape=True)
              cls_mask = ndimage.rotate(cls_mask,angle,reshape=True)

        # image = cv2.resize(image, (128,128))/255.0
        # cls_mask = cv2.resize(cls_mask, (128,128))
        image = cv2.resize(image, (patchSize,patchSize))/255.0
        cls_mask = cv2.resize(cls_mask, (patchSize,patchSize)) 
        image = np.moveaxis(image, -1, 0)

        return torch.tensor(image).float(), torch.tensor(cls_mask, dtype=torch.int64), img_path


    def __len__(self):
        return len(self.IMG_NAMES)

In [11]:
dataset = segTrainDataset('Cropped MIC dataset/TrainingData', training = True)

len(dataset)

1986

In [12]:
testDataset = segTrainDataset('Cropped MIC dataset/TestingData', training = True)

len(testDataset)

250

In [13]:
train_dataset = dataset
test_dataset = testDataset

In [14]:
BACH_SIZE = 16
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BACH_SIZE, shuffle=True, num_workers=2)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=2)

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [16]:
import torch.nn.functional as F
from torch.nn import ModuleList
import torch



class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, out_channels=32):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        bilinear = False

        self.inc = DoubleConv(n_channels, out_channels)
        self.down1 = Down(out_channels, out_channels * 2)
        self.down2 = Down(out_channels * 2, out_channels * 4)
        self.down3 = Down(out_channels * 4, out_channels * 8)
        factor = 2 if bilinear else 1
        self.down4 = Down(out_channels * 8, out_channels * 16 // factor)
        self.up1 = Up(out_channels * 16, out_channels * 8 // factor, bilinear)
        self.up2 = Up(out_channels * 8, out_channels * 4 // factor, bilinear)
        self.up3 = Up(out_channels * 4, out_channels * 2 // factor, bilinear)
        self.up4 = Up(out_channels * 2, out_channels, bilinear)
        self.outc = OutConv(out_channels, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return x1, x, logits


class MiniUNet(nn.Module):
    def __init__(self, n_channels, n_classes, out_channels=32):
        super(MiniUNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        bilinear = False

        self.inc = DoubleConv(n_channels, out_channels)
        self.down1 = Down(out_channels, out_channels*2)
        self.down2 = Down(out_channels*2, out_channels*4)
        self.down3 = Down(out_channels*4, out_channels*8)
        self.up1 = Up(out_channels*8, out_channels*4, bilinear)
        self.up2 = Up(out_channels*4, out_channels*2, bilinear)
        self.up3 = Up(out_channels*2, out_channels, bilinear)
        self.outc = OutConv(out_channels, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x = self.up1(x4, x3)
        x = self.up2(x, x2)
        x = self.up3(x, x1)
        logits = self.outc(x)
        return x1, x, logits


class Iternet(nn.Module):
    def __init__(self, n_channels, n_classes, out_channels=32, iterations=3):
        super(Iternet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.iterations = iterations

        # define the network UNet layer
        self.model_unet = UNet(n_channels=n_channels,
                               n_classes=n_classes, out_channels=out_channels)

        # define the network MiniUNet layers
        self.model_miniunet = ModuleList(MiniUNet(
            n_channels=out_channels*2, n_classes=n_classes, out_channels=out_channels) for i in range(iterations))

    def forward(self, x):
        x1, x2, logits = self.model_unet(x)
        for i in range(self.iterations):
            x = torch.cat([x1, x2], dim=1)
            _, x2, logits = self.model_miniunet[i](x)

        return logits


In [17]:
# Updated
class WeightedDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True, n_classes=2):
        super(WeightedDiceLoss, self).__init__()
        self.classes = n_classes

    def to_one_hot(self, tensor):
        n,h,w = tensor.size()
        one_hot = torch.zeros(n,self.classes,h,w).to(tensor.device).scatter_(1,tensor.view(n,1,h,w),1)
        return one_hot

    def forward(self, inputs, target):
       
        N = inputs.size()[0]
       

        # predicted probabilities for each pixel along channel
        inputs = F.softmax(inputs,dim=1)
        
        
        # Numerator Product
        target_oneHot = self.to_one_hot(target)
        weight_invertedChannels = 3
        weight_background = 1

        inter = 2*inputs * target_oneHot
        #print("inter size", inter.size())
        #(2.*intersection + smooth)/(inputs.sum() + target.sum() + smooth)
        intersection_channels = 0
        intersection_background = 0
        # smooth = 1

        for imageInd in range(N): 
          intersection_channels += inter[imageInd][1].view(-1).sum() 
          intersection_background += inter[imageInd][0].view(-1).sum() 

        # inter = intersection 
        #print("inter ", inter)
        #Denominator 
        union= inputs + target_oneHot
        ## Sum over all pixels N x C x H x W => N x C
        union_channels=0
        union_background = 0
        total2 = 0
        for imageInd in range(N):
          union_channels += union[imageInd][1].view(-1).sum() 
          union_background += union[imageInd][0].view(-1).sum() 

        # union = total2
        #print("union ", union)
        #union = union.view(N,self.classes,-1).sum(2)
        dice_channels =intersection_channels/union_channels
        dice_channels = dice_channels.mean()*0.8

        dice_background =intersection_background/union_background
        dice_background = dice_background.mean()*0.2

        weighted_dice = (dice_channels+ dice_background)

        loss = 1 - weighted_dice
        #print("loss ", loss)
        #print("loss mean ", loss.mean())
        ## Return average loss over classes and batch
        # return 1-loss.mean()
        return loss

In [18]:
criterion = WeightedDiceLoss(n_classes=2).to(device)

In [19]:
def acc(label, predicted):
  seg_acc = (y.cpu() == torch.argmax(pred_mask, axis=1).cpu()).sum() / torch.numel(y.cpu())
  return seg_acc

In [20]:
min_loss = torch.tensor(float('inf'))
# model = UNet(n_channels=3, n_classes=2).to(device)
model = Iternet(n_channels=3, n_classes=2).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

In [ ]:
from pprint import pprint
os.makedirs('./saved_models', exist_ok=True)

N_EPOCHS = 30
N_DATA = len(train_dataset)
N_TEST = len(test_dataset)

plot_losses = []
scheduler_counter = 0
prediction = []

for epoch in range(N_EPOCHS):
  # training
  model.train()
  loss_list = []
  acc_list = []
  for batch_i, (x, y,z) in enumerate(train_dataloader):

      pred_mask = model(x.to(device))

      loss = criterion(pred_mask, y.to(device))

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_list.append(loss.cpu().detach().numpy())
      acc_list.append(acc(y,pred_mask).numpy())

      sys.stdout.write(
          "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f)]"
          % (
              epoch,
              N_EPOCHS,
              batch_i,
              len(train_dataloader),
              loss.cpu().detach().numpy(),
              np.mean(loss_list),
          )
      )
  scheduler_counter += 1
  # testing
  model.eval()
  val_loss_list = []
  val_acc_list = []
  for batch_i, (x, y, z) in enumerate(test_dataloader):
      with torch.no_grad():    
          pred_mask = model(x.to(device))
      val_loss = criterion(pred_mask, y.to(device))
      val_loss_list.append(val_loss.cpu().detach().numpy())
      val_acc_list.append(acc(y,pred_mask).numpy())
    
  print(' epoch {} - loss : {:.5f} - acc : {:.2f} - val loss : {:.5f} - val acc : {:.2f}'.format(epoch, 
                                                                                                 np.mean(loss_list), 
                                                                                                 np.mean(acc_list), 
                                                                                                 np.mean(val_loss_list),
                                                                                                 np.mean(val_acc_list)))
  plot_losses.append([epoch, np.mean(loss_list), np.mean(val_loss_list)])
  #  print("prediction ", prediction.size(), prediction.data[0])
  compare_loss = np.mean(val_loss_list)
  is_best = compare_loss < min_loss
  if is_best == True:
    scheduler_counter = 0
    min_loss = min(compare_loss, min_loss)
  torch.save(model.state_dict(), './saved_models/unet_epoch_{}_{:.5f}.pt'.format(epoch,np.mean(val_loss_list)))
  
  if scheduler_counter > 5:
    lr_scheduler.step()
    print(f"lowering learning rate to {optimizer.param_groups[0]['lr']}")
    scheduler_counter = 0
